In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate

In [2]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

# Data creation

In [3]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
    display(pret)

Hilbert matrices of order 4, 5

In [4]:
h4=linalg.hilbert(4)
pretty_view(h4)

,1,2,3,4
1,1.00000000,0.50000000,0.33333333,0.25000000
2,0.50000000,0.33333333,0.25000000,0.20000000
3,0.33333333,0.25000000,0.20000000,0.16666667
4,0.25000000,0.20000000,0.16666667,0.14285714


In [5]:
h5=linalg.hilbert(5)
pretty_view(h5)

,1,2,3,4,5
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111


Diagonally dominant matrix

# Methods realisation

In [6]:
def power_method(A, eps):
    x_0 = np.array(np.ones(A.shape[0]))
    max_eig = 0
    n_iter = 0
    while True:
        n_iter += 1
        x_1 = np.dot(A, x_0)
        if abs(x_1[0] / x_0[0] - max_eig) < eps:
            max_eig = x_1[0] / x_0[0]
            break
        max_eig = x_1[0] / x_0[0]
        x_0 = x_1
    return max_eig, n_iter

In [7]:
def scalar_method(A, eps):
    x_0 = np.array(np.ones(A.shape[0]))
    y_0 = np.array(np.ones(A.shape[0]))
    max_eig = 0
    n_iter = 0
    while True:
        n_iter += 1
        x_1 = np.dot(A, x_0)
        y_1 = np.dot(A.T, y_0)
        if abs(np.dot(x_1, y_1) / np.dot(x_0, y_1) - max_eig) < eps:
            max_eig = np.dot(x_1, y_1) / np.dot(x_0, y_1)
            break
        max_eig = np.dot(x_1, y_1) / np.dot(x_0, y_1)
        x_0 = x_1
        y_0 = y_1
    return max_eig, n_iter

# Print results

In [8]:
def print_result(A):
    pretty_view(A)
    np_max_eig = max(abs(np.linalg.eig(A)[0]))
    table=[['Приближение', "n_iter степенной","n_iter скалярный", 
                'np_eig - pow_eig', 'np_eig - scal_eig']]
    for eps in (1e-2, 1e-3, 1e-4, 1e-5):
        row=[]
        row.append(eps)
        pow_eig, n_iter_1 = power_method(A, eps)
        scal_eig, n_iter_2 = scalar_method(A, eps)
        row.append(n_iter_1)
        row.append(n_iter_2)
        row.append(np.linalg.norm(np_max_eig - pow_eig))
        row.append(np.linalg.norm(np_max_eig- scal_eig))
        table.append(row)
    return print(tabulate(table, tablefmt='fancy_grid', numalign="right"))

In [9]:
list_of_matrixes=[h4, h5]

In [10]:
for matrix in list_of_matrixes:
    print_result(matrix)

,1,2,3,4
1,1.00000000,0.50000000,0.33333333,0.25000000
2,0.50000000,0.33333333,0.25000000,0.20000000
3,0.33333333,0.25000000,0.20000000,0.16666667
4,0.25000000,0.20000000,0.16666667,0.14285714


╒═════════════╤══════════════════╤══════════════════╤═══════════════════════╤═══════════════════════╕
│ Приближение │ n_iter степенной │ n_iter скалярный │ np_eig - pow_eig      │ np_eig - scal_eig     │
├─────────────┼──────────────────┼──────────────────┼───────────────────────┼───────────────────────┤
│ 0.01        │ 4                │ 3                │ 0.000625661090738916  │ 4.832066194504492e-06 │
├─────────────┼──────────────────┼──────────────────┼───────────────────────┼───────────────────────┤
│ 0.001       │ 5                │ 3                │ 7.051089044129988e-05 │ 4.832066194504492e-06 │
├─────────────┼──────────────────┼──────────────────┼───────────────────────┼───────────────────────┤
│ 0.0001      │ 6                │ 4                │ 7.94935753334336e-06  │ 6.142239605821942e-08 │
├─────────────┼──────────────────┼──────────────────┼───────────────────────┼───────────────────────┤
│ 1e-05       │ 7                │ 4                │ 8.962432478387683e-07 │ 6.14

,1,2,3,4,5
1,1.00000000,0.50000000,0.33333333,0.25000000,0.20000000
2,0.50000000,0.33333333,0.25000000,0.20000000,0.16666667
3,0.33333333,0.25000000,0.20000000,0.16666667,0.14285714
4,0.25000000,0.20000000,0.16666667,0.14285714,0.12500000
5,0.20000000,0.16666667,0.14285714,0.12500000,0.11111111


╒═════════════╤══════════════════╤══════════════════╤════════════════════════╤════════════════════════╕
│ Приближение │ n_iter степенной │ n_iter скалярный │ np_eig - pow_eig       │ np_eig - scal_eig      │
├─────────────┼──────────────────┼──────────────────┼────────────────────────┼────────────────────────┤
│ 0.01        │ 4                │ 3                │ 0.001221369082482715   │ 1.3406487584743942e-05 │
├─────────────┼──────────────────┼──────────────────┼────────────────────────┼────────────────────────┤
│ 0.001       │ 6                │ 3                │ 2.1610229907320644e-05 │ 1.3406487584743942e-05 │
├─────────────┼──────────────────┼──────────────────┼────────────────────────┼────────────────────────┤
│ 0.0001      │ 7                │ 4                │ 2.875727316054011e-06  │ 2.3741478982763908e-07 │
├─────────────┼──────────────────┼──────────────────┼────────────────────────┼────────────────────────┤
│ 1e-05       │ 8                │ 5                │ 3.82684781